In [1]:
%matplotlib inline

import numpy as np
import scipy
import scipy.io
import scipy.sparse as sp
from itertools import groupby
import pandas as pd
import warnings
warnings.simplefilter("ignore")
import os
from helpers import *

In [2]:
# train_dataset = "./data/data_train.csv"
# ratings = load_data(train_dataset)

number of items: 10000, number of users: 1000


In [2]:
def read_txt(path):
    """read text file from path."""
    with open(path, "r") as f:
        return f.read().splitlines()


def load_data(path_dataset):
    """Load data in text format, one rating per line, as in the kaggle competition."""
    data = read_txt(path_dataset)[1:]
    return preprocess_data(data)


def preprocess_data(data):
    """preprocessing the text data, conversion to numerical array format."""
    def deal_line(line):
        pos, rating = line.split(',')
        row, col = pos.split("_")
        row = row.replace("r", "")
        col = col.replace("c", "")
        return int(row), int(col), float(rating)

    def statistics(data):
        row = set([line[0] for line in data])
        col = set([line[1] for line in data])
        return min(row), max(row), min(col), max(col)

    # parse each line
    data = [deal_line(line) for line in data]

    # do statistics on the dataset.
    min_row, max_row, min_col, max_col = statistics(data)
    print("number of items: {}, number of users: {}".format(max_row, max_col))

    # build rating matrix.
    ratings = sp.lil_matrix((max_row, max_col))
    for row, col, rating in data:
        ratings[row - 1, col - 1] = rating
    return ratings


def group_by(data, index):
    """group list of list by a specific index."""
    sorted_data = sorted(data, key=lambda x: x[index])
    groupby_data = groupby(sorted_data, lambda x: x[index])
    return groupby_data


def build_index_groups(train):
    """build groups for nnz rows and cols."""
    # row : items; cols: users
    nz_row, nz_col = train.nonzero()
    nz_train = list(zip(nz_row, nz_col))

    grouped_nz_train_byrow = group_by(nz_train, index=0) # group by items 
#     for g, value in grouped_nz_train_byrow:
#         print("{}, {}".format(g, list(value))) #value for g=0: (0, 1) (0, 2) (0, 3) index of all the users that rated the item 0
    nz_row_colindices = [(g, np.array([v[1] for v in value])) # indices of all the users that rated item g
                         for g, value in grouped_nz_train_byrow]
    
#     print(nz_row_colindices)

    grouped_nz_train_bycol = group_by(nz_train, index=1) # group by users
    nz_col_rowindices = [(g, np.array([v[0] for v in value])) # indices of all the movies rated by user g
                         for g, value in grouped_nz_train_bycol]
    return nz_train, nz_row_colindices, nz_col_rowindices

In [3]:
def get_number_per(ratings):
    """plot the statistics result on raw rating data."""
    # do statistics.
    num_items_per_user = np.array((ratings != 0).sum(axis=0)).flatten()
    num_users_per_item = np.array((ratings != 0).sum(axis=1).T).flatten()
    sorted_num_movies_per_user = np.sort(num_items_per_user)[::-1]
    sorted_num_users_per_movie = np.sort(num_users_per_item)[::-1]
    return num_items_per_user, num_users_per_item

In [4]:
# num_items_per_user,num_users_per_item = get_number_per(ratings)

In [4]:
def split_data(ratings, num_items_per_user, num_users_per_item,
               min_num_ratings, p_test=0.1):
    """split the ratings to training data and test data.
    Args:
        min_num_ratings: 
            all users and items we keep must have at least min_num_ratings per user and per item. 
    """
#     # set seed
#     np.random.seed(988)
    
    # select user and item based on the condition.
    valid_users = np.where(num_items_per_user >= min_num_ratings)[0]
    valid_items = np.where(num_users_per_item >= min_num_ratings)[0]
    valid_ratings = ratings[valid_items, :][: , valid_users]  
    
    # init
    num_rows, num_cols = valid_ratings.shape
    train = sp.lil_matrix((num_rows, num_cols))
    test = sp.lil_matrix((num_rows, num_cols))
    
    print("the shape of original ratings. (# of row, # of col): {}".format(
        ratings.shape))
    print("the shape of valid ratings. (# of row, # of col): {}".format(
        (num_rows, num_cols)))

    nz_items, nz_users = valid_ratings.nonzero()
    
    # split the data
    for user in set(nz_users):
        # randomly select a subset of ratings
        row, col = valid_ratings[:, user].nonzero()
        selects = np.random.choice(row, size=int(len(row) * p_test))
        residual = list(set(row) - set(selects))

        # add to train set
        train[residual, user] = valid_ratings[residual, user]

        # add to test set
        test[selects, user] = valid_ratings[selects, user]
    
    
    print("Total number of nonzero elements in origial data:{v}".format(v=ratings.nnz))
    print("Total number of nonzero elements in train data:{v}".format(v=train.nnz))
    print("Total number of nonzero elements in test data:{v}".format(v=test.nnz))
    return valid_ratings, train, test

In [6]:
# valid_ratings, train, test = split_data(
#     ratings, num_items_per_user, num_users_per_item, min_num_ratings=0, p_test=0.2)

the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:941957
Total number of nonzero elements in test data:234995


In [7]:
# type(test)

scipy.sparse.lil.lil_matrix

In [5]:
def init_MF(train, num_features,weight=1.0):
    """init the parameter for matrix factorization."""
    
    num_item,num_user = train.shape
    
    user_features = weight * np.random.rand(num_features,num_user)
    item_features = weight * np.random.rand(num_features,num_item)
    
    item_nnz = train.getnnz(axis=1)
    item_sum = train.sum(axis=1)
    
    return user_features, item_features

In [6]:
def compute_error(data, user_features, item_features, nz):
    """compute the loss (MSE) of the prediction of nonzero elements."""
    # calculate rmse (we only consider nonzero entries.)
    mse = 0
    for row,col in nz:
        user = user_features[:,col]
        item = item_features[:,row]
        mse += ((data[row,col] - user.T.dot(item))**2)
    
    rmse = np.sqrt(1.0*mse/len(nz))
    return rmse

In [7]:
def update_user_feature(
        train, item_features, lambda_user,
        nnz_items_per_user, nz_user_itemindices):
    """update user feature matrix."""
    """the best lambda is assumed to be nnz_items_per_user[user] * lambda_user"""
    # update and return user feature.
    num_users = nnz_items_per_user.shape[0]
    num_features = item_features.shape[0]
    lambda_I = lambda_user * sp.eye(num_features)
    updated_user_features = np.zeros((num_features,num_users))
    
    for user,item in nz_user_itemindices:
        M = item_features[:,item]
        
        V = M @ train[item,user]
        A = M @ M.T + nnz_items_per_user[user] * lambda_I
        Z_star = np.linalg.solve(A,V)
        updated_user_features[:,user] = np.copy(Z_star.T)
    return updated_user_features

def update_item_feature(
        train, user_features, lambda_item,
        nnz_users_per_item, nz_item_userindices):
    """update item feature matrix."""
    """the best lambda is assumed to be nnz_items_per_item[item] * lambda_item"""
    # update and return item feature.
    num_items = nnz_users_per_item.shape[0]
    num_features = user_features.shape[0]
    lambda_I = lambda_item * sp.eye(num_features)
    updated_item_features = np.zeros((num_features,num_items))
    
    for item,user in nz_item_userindices:
        M = user_features[:,user]
        
        V = M @ train[item,user].T
        A = M @ M.T + nnz_users_per_item[item] * lambda_I
        W_star = np.linalg.solve(A,V)
        updated_item_features[:,item] = np.copy(W_star.T)
    return updated_item_features

In [8]:
def ALS(train, test,num_features,lambda_user,lambda_item,max_weight=1.0,iterations=50):
    """Alternating Least Squares (ALS) algorithm."""
    # define parameters
    stop_criterion = 1e-5
    change = 1
    error_list = [0, 0]
    it = 0
 
    nz_row, nz_col = train.nonzero()
    nz_train = list(zip(nz_row, nz_col))
    
    nz_row, nz_col = test.nonzero()
    nz_test = list(zip(nz_row, nz_col))
    
#     user_features_file_path = './data/user_features_%s_%s_%s_%s.npy' \
#         % (iterations, num_features, lambda_user, lambda_item)

#     item_features_file_path = './data/item_features_%s_%s_%s_%s.npy' \
#         % (iterations, num_features, lambda_user, lambda_item)
    
#     if(os.path.exists(user_features_file_path) and os.path.exists(item_features_file_path)):
#         user_features = np.load(user_features_file_path)
#         item_features = np.load(item_features_file_path)

#         train_rmse = compute_error(train, user_features, item_features,nz_train)

#         test_rmse = compute_error(test, user_features, item_features,nz_test)
        
#         print("Train RMSE: {tr_rmse}, test RMSE: {te_rmse}" .format(tr_rmse=train_rmse, te_rmse=test_rmse))

#         return user_features, item_features,test_rmse
    
    # set seed
#     np.random.seed(988)

    # init ALS
    user_features, item_features = init_MF(train, num_features,max_weight)
    
    # get the number of non-zero ratings for each user and item
    nnz_items_per_user,nnz_users_per_item = train.getnnz(axis=0),train.getnnz(axis=1)
    
    # group the indices by row or column index
    _, nz_item_userindices, nz_user_itemindices = build_index_groups(train)
    
    train_rmse = 0
    # start ALS
    while(it < iterations):
        user_features = update_user_feature(train, item_features, lambda_user,
                            nnz_items_per_user, nz_user_itemindices)
        
        item_features = update_item_feature(train, user_features, lambda_item,
                            nnz_users_per_item, nz_item_userindices)
        
        train_rmse = compute_error(train,user_features,item_features,nz_train)
#         print("ALS training RMSE : {err}".format(err=train_rmse))
        error_list.append(train_rmse)
        change = np.fabs(error_list[-1] - error_list[-2])
        if (change < stop_criterion):
            print("Converge!")
            break;
        it += 1
        
    print("ALS Final training RMSE : {err}".format(err=train_rmse))
    # evaluate the error in test set
    
    test_rmse = compute_error(test, user_features, item_features, nz_test)
    print("RMSE on test data after ALS: {}.".format(test_rmse))   
    
#     np.save(user_features_file_path, user_features)
#     np.save(item_features_file_path, item_features)
    
    return item_features,user_features,test_rmse

In [17]:
# num_features = 50
# lambda_user = 0.2
# lambda_item = 0.02
# item_features,user_features,_ = ALS(train, test,num_features,lambda_user,lambda_item)

ALS training RMSE : 0.6804265713964545
ALS training RMSE : 0.6160907382671245
ALS training RMSE : 0.5672073485553909
ALS training RMSE : 0.5497330414721812
ALS training RMSE : 0.5415995602519763
ALS training RMSE : 0.5368605368734932
ALS training RMSE : 0.5337420789679976
ALS training RMSE : 0.5315389413941832
ALS training RMSE : 0.5299072035495632
ALS training RMSE : 0.5286563915443282
ALS training RMSE : 0.527671453213674
ALS training RMSE : 0.5268784327757908
ALS training RMSE : 0.5262276284246435
ALS training RMSE : 0.5256845598481641
ALS training RMSE : 0.5252247230577611
ALS training RMSE : 0.5248303451995805
ALS training RMSE : 0.5244882944256996
ALS training RMSE : 0.524188703361501
ALS training RMSE : 0.5239240474442701
ALS training RMSE : 0.5236885153834951
ALS training RMSE : 0.5234775679682886
ALS training RMSE : 0.5232876201623021
ALS training RMSE : 0.5231158065289976
ALS training RMSE : 0.5229598055817284
ALS training RMSE : 0.5228177077655134
ALS training RMSE : 0.52268

In [55]:
# def cv_ALS_grid_search(train,test,seed=988):
#     # set seed
# #     np.random.seed(seed)
    
#     lambda_users = np.linspace(0.01,0.21,num=21)
#     lambda_items = np.linspace(0.01,0.21,num=21)
#     nb_features = 50
#     weights = np.linspace(1.0,3.0,num=30)
#     # for test
# #     lambda_users = [0.01]
# #     lambda_items = [0.2]
# #     nb_features = 20
# #     weights = [1.0]
                    
#     best_weight = -1
#     best_lambda_item = -1
#     best_lambda_user = -1
#     best_num_feature = -1
#     best_rmse = 100
    
#     newpath = r'./data' 
#     if not os.path.exists(newpath):
#         os.makedirs(newpath)
#     for i in range(20,nb_features+1):
#         num_features = i
#         for weight in weights:
#             for lambda_user in lambda_users:
#                 for lambda_item in lambda_items:
#                     item_features,user_features,test_rmse = ALS(train, test,num_features,lambda_user,lambda_item,weight)
#                     if(test_rmse < best_rmse):
#                         best_rmse = test_rmse
#                         bset_lambda_item = lambda_item
#                         best_lambda_user = lambda_user
#                         best_weight = weight
#                         best_num_feature = num_features
#                         best_rmse = test_rmse
#                         print("CHANGE=====>best rmse: {},lambda_item :{},lambda_user:{},weight:{},num_feature:{}"\
#                               .format(best_rmse,bset_lambda_item,best_lambda_user,best_weight,best_num_feature))
#     print("=======>>>>FINAL: BEST RMSE: {},lambda_item :{},lambda_user:{},weight:{},num_feature:{}"\
#                               .format(best_rmse,bset_lambda_item,best_lambda_user,best_weight,best_num_feature))
#     best_param = np.array([best_num_feature,best_weight,best_lambda_user,bset_lambda_item])
#     np.save("best_param_grid_search.npy", best_param)
#     return best_num_feature,best_weight,best_lambda_user,bset_lambda_item

In [9]:
def cv_ALS_random_search(train,test,seed=988):
#     # set seed
#     np.random.seed(seed)
    users_range = np.linspace(0.01,1,num=100)
    item_range = np.linspace(0.01,1,num=100)
#     features_num_range = 60
    features_num_range = np.linspace(1,60,num=60,dtype=np.int32)
    weight_range = np.linspace(1.0,3.0,num=60)
    
    lambda_users = np.random.choice(users_range,60)
    lambda_items = np.random.choice(item_range,60)
    nb_features = np.random.choice(features_num_range,60)
    weights = np.random.choice(weight_range,60)
    
    # for test
#     lambda_users = [0.01]
#     lambda_items = [0.2]
#     nb_features = [20]
#     weights = [1.0]
    
    best_weight = -1
    best_lambda_item = -1
    best_lambda_user = -1
    best_num_feature = -1
    best_rmse = 100
    
    k_fold = 5
#     # set seed
#     np.random.seed(seed)
    
    newpath = r'./data' 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    train_tr_list, test_tr_list = split_for_cv(train,p_test=0.2)
    for num_features,weight,lambda_user,lambda_item in zip(nb_features,weights,lambda_users,lambda_items):
        rmse_list = []
        for train_tr,test_tr in zip(train_tr_list, test_tr_list): # 5-fold cv
            item_features,user_features,test_tr_rmse = ALS(train_tr, test_tr,num_features,lambda_user,lambda_item,weight)
#             print("test RMSE: {te_rmse}" .format(te_rmse=test_tr_rmse))
            rmse_list.append(test_tr_rmse)
        test_rmse = np.mean(rmse_list)
        if(test_rmse < best_rmse):
            best_rmse = test_rmse
            bset_lambda_item = lambda_item
            best_lambda_user = lambda_user
            best_weight = weight
            best_num_feature = num_features
            best_rmse = test_rmse
            print("CHANGE=====>best rmse: {},lambda_item :{},lambda_user:{},weight:{},num_feature:{}"\
                  .format(best_rmse,bset_lambda_item,best_lambda_user,best_weight,best_num_feature))
            
    print("=======>>>> FINAL: BEST RMSE: {},lambda_item :{},lambda_user:{},weight:{},num_feature:{}"\
                              .format(best_rmse,bset_lambda_item,best_lambda_user,best_weight,best_num_feature))
    
    best_param = np.array([best_num_feature,best_weight,best_lambda_user,bset_lambda_item])
    np.save("best_param_random_search.npy", best_param)
    return best_num_feature,best_weight,best_lambda_user,bset_lambda_item

In [10]:
def split_for_cv(train,p_test=0.2,k_fold=5):
    # init
    num_rows, num_cols = train.shape
    nz_items, nz_users = train.nonzero()
    train_tr_list=[]
    test_tr_list = []
    # split the data
    for k in range(k_fold):
        train_tr = sp.lil_matrix((num_rows, num_cols))
        test_tr = sp.lil_matrix((num_rows, num_cols))
        for user in set(nz_users):
            # randomly select a subset of ratings
            row, col = train[:, user].nonzero()
            selects = np.random.choice(row, size=int(len(row) * p_test))
            residual = list(set(row) - set(selects))

            # add to train set
            train_tr[residual, user] = train[residual, user]

            # add to test set
            test_tr[selects, user] = train[selects, user]
            
        train_tr_list.append(train_tr)
        test_tr_list.append(test_tr)
        
    return train_tr_list, test_tr_list

In [85]:
def predict_ALS(train,test,num_features=None,lambda_user=None,lambda_item=None,weight=None,load_File=None):
    if(load_File==1):
        best_param = np.load("best_param_random_search.npy")
        num_features = best_param[0]
        weight = best_param[1]
        lambda_user = best_param[2]
        lambda_item = best_param[3]
    
    item_features,user_features , _ = ALS(train, test,num_features,lambda_user,lambda_item,weight)
    predict_labels = item_features.T @ user_features
    predict_labels[predict_labels > 5] = 5
    predict_labels[predict_labels < 1] = 1
    
    predict = np.asarray(predict_labels)
    np.save("predict.npy",predict)
    
    return predict
    # Generate the CSV submission file
    #generate_submission(predicted_labels)

In [92]:
def run(intest=0,seed=988):
    train_dataset = "./data/data_train.csv"
    ratings = load_data(train_dataset)
    
    # set seed
    np.random.seed(seed)
    
    num_items_per_user,num_users_per_item = get_number_per(ratings)
    valid_ratings, train, test = split_data(
    ratings, num_items_per_user, num_users_per_item, min_num_ratings=0, p_test=0.2)
    if(intest ==1):
        num_features = 20
        weight = 1.0
        lambda_user = 0.2
        lambda_item = 0.02
    else:
        num_features,weight,lambda_user,lambda_item = cv_ALS_random_search(train,test)
#     num_features,weight,lambda_user,lambda_item = cv_ALS_grid_search(train,test)
#     predict_ALS(train,test,load_File=1)
#     np.random.seed(988)
    predict_ALS(train,test,num_features,lambda_user,lambda_item,weight)

## Test Function

In [34]:
#test function
run(1)

number of items: 10000, number of users: 1000
the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:941957
Total number of nonzero elements in test data:234995
ALS training RMSE : 0.9767730275311427
RMSE on test data after ALS: 1.001994343538817.


In [69]:
train_dataset = "./data/data_train.csv"
ratings = load_data(train_dataset)
num_items_per_user,num_users_per_item = get_number_per(ratings)
valid_ratings, train, test = split_data(
ratings, num_items_per_user, num_users_per_item, min_num_ratings=0, p_test=0.2)

number of items: 10000, number of users: 1000
the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:963742
Total number of nonzero elements in test data:213210


In [81]:
num_features,weight,lambda_user,lambda_item = cv_ALS_random_search(train,test)

ALS Final training RMSE : 0.8249949573586155
RMSE on test data after ALS: 0.8242937198559628.
test RMSE: 0.8242937198559628
ALS Final training RMSE : 0.8252324088835039
RMSE on test data after ALS: 0.8247501770710333.
test RMSE: 0.8247501770710333
ALS Final training RMSE : 0.8251761616150919
RMSE on test data after ALS: 0.824538177162649.
test RMSE: 0.824538177162649
ALS Final training RMSE : 0.8253676057344289
RMSE on test data after ALS: 0.8244818615498287.
test RMSE: 0.8244818615498287
ALS Final training RMSE : 0.8252385791149907
RMSE on test data after ALS: 0.8243587954515158.
test RMSE: 0.8243587954515158
CHANGE=====>best rmse: 0.824484546218198,lambda_item :0.2,lambda_user:0.01,weight:1.0,num_feature:20
=======>>>> FINAL: BEST RMSE: 0.824484546218198,lambda_item :0.2,lambda_user:0.01,weight:1.0,num_feature:20


In [54]:
num_features,weight,lambda_user,lambda_item = cv_ALS_grid_search(train,test)

Train RMSE: 1.0085226256030653, test RMSE: 1.0327025362264135
CHANGE=====>best rmse: 1.0327025362264135,lambda_item :0.2,lambda_user:0.01,weight:1.0,num_feature:20
=======>>>>FINAL: BEST RMSE: 1.0327025362264135,lambda_item :0.2,lambda_user:0.01,weight:1.0,num_feature:20


## Run random search

In [62]:
run()

number of items: 10000, number of users: 1000
the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:941957
Total number of nonzero elements in test data:234995
Converge!
ALS Final training RMSE : 1.2844089848348634
RMSE on test data after ALS: 1.2919185835400455.
CHANGE=====>best rmse: 1.2919185835400455,lambda_item :0.9400000000000001,lambda_user:0.7100000000000001,weight:2.9661016949152543,num_feature:32
Converge!
ALS Final training RMSE : 1.0810079018281236
RMSE on test data after ALS: 1.0906493535590387.
CHANGE=====>best rmse: 1.0906493535590387,lambda_item :0.5700000000000001,lambda_user:0.33,weight:2.3220338983050848,num_feature:34
Converge!
ALS Final training RMSE : 1.1279576635076167
RMSE on test data after ALS: 1.1370043252059976.
Converge!
ALS Final training RMSE : 1.0872603753772792
RMSE on test d

### After modifying cross validation

In [95]:
run()

number of items: 10000, number of users: 1000
the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:963742
Total number of nonzero elements in test data:213210
Converge!
ALS Final training RMSE : 1.3717441129343997
RMSE on test data after ALS: 1.3787216172274093.
Converge!
ALS Final training RMSE : 1.3719836828840728
RMSE on test data after ALS: 1.3771756192963431.
Converge!
ALS Final training RMSE : 1.3716211007834207
RMSE on test data after ALS: 1.3810476885851843.
Converge!
ALS Final training RMSE : 1.371656746775496
RMSE on test data after ALS: 1.3804117114326881.
Converge!
ALS Final training RMSE : 1.3717704761187886
RMSE on test data after ALS: 1.3810379518481368.
CHANGE=====>best rmse: 1.3796789176779523,lambda_item :0.9,lambda_user:1.0,weight:1.2033898305084745,num_feature:46
Converge!
ALS Final trai

RMSE on test data after ALS: 0.994391042085989.
ALS Final training RMSE : 0.8823597221463976
RMSE on test data after ALS: 0.9953297193337383.
ALS Final training RMSE : 0.882561263025589
RMSE on test data after ALS: 0.9946929362172725.
CHANGE=====>best rmse: 0.9942181947494481,lambda_item :0.47000000000000003,lambda_user:0.02,weight:2.1864406779661016,num_feature:20
ALS Final training RMSE : 0.7888157640965784
RMSE on test data after ALS: 1.0024889438794549.
ALS Final training RMSE : 0.7892524800546797
RMSE on test data after ALS: 1.0009379081076568.
ALS Final training RMSE : 0.7888493387029238
RMSE on test data after ALS: 1.003050101822153.
ALS Final training RMSE : 0.7890895452166216
RMSE on test data after ALS: 1.0037604482509932.
ALS Final training RMSE : 0.7890850012752084
RMSE on test data after ALS: 1.0035713010534035.
Converge!
ALS Final training RMSE : 1.1598784543255796
RMSE on test data after ALS: 1.1691969296706228.
Converge!
ALS Final training RMSE : 1.1601687327851786
RMSE

RMSE on test data after ALS: 1.3042247345856814.
Converge!
ALS Final training RMSE : 1.2942085124200797
RMSE on test data after ALS: 1.3035879434992563.
Converge!
ALS Final training RMSE : 1.2943273868802276
RMSE on test data after ALS: 1.304132661810086.
Converge!
ALS Final training RMSE : 1.0571311020863674
RMSE on test data after ALS: 1.0681661531921085.
Converge!
ALS Final training RMSE : 1.0574533586252757
RMSE on test data after ALS: 1.0664119351746668.
Converge!
ALS Final training RMSE : 1.0569806137212305
RMSE on test data after ALS: 1.0696669999167603.
Converge!
ALS Final training RMSE : 1.0570225082000355
RMSE on test data after ALS: 1.0691228355193025.
Converge!
ALS Final training RMSE : 1.0571625980808363
RMSE on test data after ALS: 1.0692171303931108.
Converge!
ALS Final training RMSE : 1.1258296840318454
RMSE on test data after ALS: 1.1356501314016338.
Converge!
ALS Final training RMSE : 1.126129896336141
RMSE on test data after ALS: 1.133925266389459.
Converge!
ALS Fina

Converge!
ALS Final training RMSE : 1.0429085613609677
RMSE on test data after ALS: 1.0552424793556556.
Converge!
ALS Final training RMSE : 1.0430502747878714
RMSE on test data after ALS: 1.055285927012656.
Converge!
ALS Final training RMSE : 1.2252335771956464
RMSE on test data after ALS: 1.2337094017332895.
Converge!
ALS Final training RMSE : 1.2255061943281227
RMSE on test data after ALS: 1.232052318672762.
Converge!
ALS Final training RMSE : 1.2250999964542377
RMSE on test data after ALS: 1.235788210304894.
Converge!
ALS Final training RMSE : 1.22513825452332
RMSE on test data after ALS: 1.235157863897624.
Converge!
ALS Final training RMSE : 1.2252621923450455
RMSE on test data after ALS: 1.235612448486516.
Converge!
ALS Final training RMSE : 1.0857791782090034
RMSE on test data after ALS: 1.0962663715737886.
Converge!
ALS Final training RMSE : 1.086091922712234
RMSE on test data after ALS: 1.0945210083022094.
Converge!
ALS Final training RMSE : 1.0856320466134841
RMSE on test data

In [13]:
def predict_ALS(num_features=None,lambda_user=None,lambda_item=None,weight=None,load_File=None):
    seed = 988
    train_dataset = "./data/data_train.csv"
    ratings = load_data(train_dataset)
    
    # set seed
    np.random.seed(seed)
    
    num_items_per_user,num_users_per_item = get_number_per(ratings)
    valid_ratings, train, test = split_data(
    ratings, num_items_per_user, num_users_per_item, min_num_ratings=0, p_test=0.1)
    
    if(load_File==1):
        best_param = np.load("best_param_random_search.npy")
        num_features = best_param[0]
        weight = best_param[1]
        lambda_user = best_param[2]
        lambda_item = best_param[3]
#     else:
#         num_features = 20
#         weight = 2.18644068
#         lambda_user = 0.02
#         lambda_item = 0.47
    item_features,user_features , _ = ALS(train, test,num_features,lambda_user,lambda_item,weight)
    predict_labels = item_features.T @ user_features
    predict = np.asarray(predict_labels.T)
    moive_user_predict = pd.DataFrame(data=predict)
    moive_user_predict.reset_index(inplace=True)
    moive_user_predict.rename(columns={"index":"Movie"},inplace=True)
    moive_user_predict_melt = pd.melt(moive_user_predict,id_vars=["Movie"],var_name="User",value_name ="Rating")
    moive_user_predict_melt["Movie"] = moive_user_predict_melt["Movie"].values +1
    moive_user_predict_melt["User"] = moive_user_predict_melt["User"].values +1
    
    sample = pd.read_csv("./data/sampleSubmission.csv")
    moive_user_predict_melt['Id'] = moive_user_predict_melt.apply(lambda x: 'r{}_c{}'.format(int(x.User), int(x.Movie)), axis=1)
    prediction = moive_user_predict_melt[moive_user_predict_melt.Id.isin(sample.Id.values)]
    prediction = prediction[["User","Movie","Rating"]]
    
    return prediction

In [23]:
sample = pd.read_csv("./data/sampleSubmission.csv")
moive_user_predict_melt['Id'] = moive_user_predict_melt.apply(lambda x: 'r{}_c{}'.format(int(x.User), int(x.Movie)), axis=1)
prediction = moive_user_predict_melt[moive_user_predict_melt.Id.isin(sample.Id.values)]
prediction = prediction[["User","Movie","Rating"]]

In [14]:
moive_user_predict_melt = predict_ALS(num_features = 20, weight = 2.18644068,lambda_user = 0.02,lambda_item = 0.47)

number of items: 10000, number of users: 1000
the shape of original ratings. (# of row, # of col): (10000, 1000)
the shape of valid ratings. (# of row, # of col): (10000, 1000)
Total number of nonzero elements in origial data:1176952
Total number of nonzero elements in train data:1065327
Total number of nonzero elements in test data:111625
ALS Final training RMSE : 0.9167933066275361
RMSE on test data after ALS: 0.9874095570030827.
